In [ ]:
import os
import pandas as pd 
from pathlib import Path

In [ ]:
cwd_path = Path('/home/apollo/code/mproj7205')
os.chdir(cwd_path)

In [ ]:
from utils.run_utils import *
from utils.report import *

import seaborn as sns

%load_ext autoreload
%autoreload 2

In [ ]:
order_of_patient_ids = [3115, 3570, 3551, 3151, 3804, 3605, 3588, 3113, 3191, \
                        4030, 3584, 3614, 3603, 3392, 3390,  3805, 3612, 3638,\
                        4037, 3126, 3374, 3571, 3589, 4010, 4032, 3621,3373, \
                        3765, 3300, 3812, 3102, 3564, 3385, 3556]

patient_ids = order_of_patient_ids

# create_images = False
#load_from_file = True

create_images = True
load_from_file = False

#### Folder structure

Inside report_results:

```
tex_from_python
├── figs
├── images
├────────── 3300
├─────────────── happy_sweep_1.png
├─────────────── happy_sweep_1.tex
├─────────────── T2_T1.png
├─────────────── baseline.png
├─────────────── baseline.tex
├─────────────── T2.png
└── tables
```

In [ ]:
df = create_project_df(project_name='projectB')

In [ ]:
df_box = compute_metrics_by_df(df)

In [ ]:
df['augmentation'] = df['augmentation'].fillna(value=False)
replace_dict = {False: 'No augmentation', True : 'Augmentation'}
df['augmentation_'] = df['augmentation'].replace(replace_dict)

In [ ]:
df['network_name'].unique()

In [ ]:
df['network_name_'].unique()

In [ ]:
df['loss_function'].unique()

In [ ]:
df['augmentation'].unique()

In [ ]:
df['augmentation_'].unique()

In [ ]:
df = df.drop_duplicates(subset=['network_name', 'loss_function', 'include_T1', 'augmentation'], keep='first').copy()

In [ ]:
df[df['augmentation']==False].shape[0]

In [ ]:
df[df['augmentation']==True].shape[0]

## Sort by names and give index

In [ ]:
df.columns

In [ ]:
df = df.sort_values(['network_name', 'loss_function', 'augmentation', 'include_T1'])

In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
#df_results = df_results[['network_name_', 'loss_function_', 'include_T1_', 'augmentation_', 'mean dsc', 'median dsc', 'mean ppv', 'mean sensitivity']]
#df.index.name = 'Exp. Id'

In [ ]:
df

## Add metrics

For each selected experiement in the dataframe, run the model on the validation set and get individual results (DSC, TP, etc) and store in df_box. 

In [ ]:
# df_box = compute_metrics_by_df(df, load_from_file=load_from_file)

In [ ]:
df =append_mean_median(df, df_box)

In [ ]:
df = df.sort_values(['mean dsc'], ascending=False)

# Print summary table

In [ ]:
df_results = df[['network_name', 'loss_function', 'include_T1', 'augmentation', 'mean dsc', 'median dsc', 'mean ppv', 'mean sensitivity']].sort_index(ascending=True)

In [ ]:
df_results = map_columns(df_results)

In [ ]:
df_results

# Create images

For each experiment in the dataframe, create the pngs for each image in the validation set

## Labeller compare

In [ ]:
if create_images:
    create_baseline_images(patent_ids=order_of_patient_ids, crop_shape=(160, 160))

In [ ]:
if create_images:
    create_experiment_images(df=df, patent_ids=order_of_patient_ids, crop_shape=(160, 160))

In [ ]:
if create_images:
    create_reference_images(patent_ids=order_of_patient_ids, crop_shape=(160,160))

In [ ]:
short_caption = 'Comparison of two different labellers for selected MRIs from the PPMI dataset.'
caption = 'Comparison of two different labellers for selected MRIs from the PPMI dataset. ' +\
          'Considering labeller one as the ground truth and labeller two as the prediction, the DSC ' + \
          'achieved by labeller two is shown along with the number of true positives/false negatives/false positives ' +\
          ' (in brackets).' + \
          'The top row shows examples with the best agreement between the two labellers. ' + \
          'The middle row shows examples with a medium level of agreement between the two labellers (near the median DSC). ' + \
          'The bottom row shows examples with the least agreement between the two labellers. '


fname_stem = f'compare_labellers_baseline_9x9'
create_3col_baseline_figure(patient_ids=order_of_patient_ids, fname_stem=fname_stem, caption=caption, short_caption=short_caption)

In [ ]:
short_caption = 'Comparison of two different labellers for selected MRIs from the PPMI dataset.'

caption = 'Comparison of two different labellers for selected MRIs from the PPMI dataset. ' +\
          'The first column shows the T2-weighted image that the labeller uses for segmentation. ' + \
          'The second column shows the corresponding coregistered T1-weighted image. ' + \
          'The third column shows the region of overlap (green) achieved by labeller two with respect to labeller one. ' + \
          'Considering labeller one as the ground truth and labeller two as the prediction, the overlap ' + \
          'can be treated as the true positives, while the region labelled only by labeller one (orange) can be considered ' +\
          'to be the false negatives and  the region labelled only by labeller two (red) can be considered the false positives.' 
#            + \
#           'The first row shows the example with the best agreement between the two labellers (highest DSC). ' + \
#           'The middle row shows the example with the median DSC. ' + \
#           'The bottom row shows the example with the least agreement between the two labellers (lowest DSC).'

selected_ = [3115, 3570, 3551]
fname_stem = f'compare_labellers_reference_9x9'
create_3col_reference_figure(patient_ids=selected_, fname_stem=fname_stem, caption=caption, short_caption=short_caption)


fname_stem = f'compare_labellers_reference_9x9_b'
create_3col_reference_figure(patient_ids=[3390, 3374, 3571], fname_stem=fname_stem, caption=caption, short_caption=short_caption)


## Create an image fore all (for selecting good results)

### Labeller baseline images

In [ ]:
idx = 0
while True:
    if (idx+9)>len(order_of_patient_ids):
        ids_subset = order_of_patient_ids[-9:]
    else:
        ids_subset = order_of_patient_ids[idx:idx+9]
        
    caption = ' '.join([str(x) for x in ids_subset])    
    fname_stem = f'compare_labellers_baseline_9x9_{str(idx)}'
    idx+=9
    
    create_3col_baseline_figure(patient_ids=ids_subset, fname_stem=fname_stem, caption=caption)  

    if idx > len(order_of_patient_ids):
        break
    

### Labeller reference images

In [ ]:
idx = 0
while True:
    if (idx+3)>len(order_of_patient_ids):
        break
        
    if (idx+3)>len(order_of_patient_ids):
        ids_subset = order_of_patient_ids[-3:]
    else:
        ids_subset = order_of_patient_ids[idx:idx+3]
        
    caption = ' '.join([str(x) for x in ids_subset])    
    fname_stem = f'compare_labellers_reference_9x9_{str(idx)}'
    idx+=3
    
    create_3col_reference_figure(patient_ids=ids_subset, fname_stem=fname_stem, caption=caption)
    
    if idx > len(order_of_patient_ids):
        break


# Create histogram for report

### Labeller histogram

In [ ]:
df_labellers = get_compare_labellers_df(verbose=True)

In [ ]:
# df_labellers = get_compare_labellers_df_v3(verbose=True)

caption = 'Density plot of DSC between two labellers over 155 MRIs'


plot_from_df(df=df_labellers, \
             fname_stem='label_uncertainty_histplot_dsc',
             caption=caption,
             set_order=False,
             x_column='DSC', \
             x_label='DSC', y_label='Density', \
             sns_plot='sns.histplot', show_annotations=False)



In [ ]:
df_labellers['DSC'].mean()

In [ ]:
df_labellers['PPV'].mean()

In [ ]:
df_labellers['Sensitivity'].median()

In [ ]:
caption = 'Density plot of DSC, PPV and sensitivity between two labellers over 155 MRIs'


plot_from_kde_df(df=df_labellers, \
                 fname_stem='label_uncertainty_histplot_all',
                 caption=caption)



In [ ]:
# df_labellers = get_compare_labellers_df_v3(verbose=True)

caption = 'Density plot of PPV between two labellers over 155 MRIs'


plot_from_df(df=df_labellers, \
             fname_stem='label_uncertainty_histplot_ppv',
             caption=caption,
             set_order=False,
             x_column='PPV', \
             x_label='PPV', y_label='Density', \
             sns_plot='sns.histplot', show_annotations=False)



In [ ]:
# df_labellers = get_compare_labellers_df_v3(verbose=True)

caption = 'Density plot of sensitivity between two labellers over 155 MRIs'


plot_from_df(df=df_labellers, \
             fname_stem='label_uncertainty_histplot_sensitivity',
             caption=caption,
             set_order=False,
             x_column='Sensitivity', \
             x_label='Sensitivity', y_label='Density', \
             sns_plot='sns.histplot', show_annotations=False)



# Get individual experiment results

We have the following experiment results to collect

* Loss functions
* Networks
* Augmentation
* Pretrained
* F-beta

We can create a filter for each type to get a df sub dataframe of the correct experiments

In [ ]:
def get_filtered_df(df):
    conditions =  ( df['loss_function'] == 'DiceLoss'    ) & \
                  ( df['network_name'] == 'UNet'         ) & \
                  ( df['augmentation'] == False          ) & \
                  ( True )
    return df[conditions]

# For example
get_filtered_df(df)

# Loss functions experiment

In [ ]:
def loss_filter(df, selected=[]):
    conditions =  ( df['network_name'] == 'UNet'         ) & \
                  ( df['augmentation'] == False          ) & \
                  ( True )
    df = df[conditions]
    if selected:
        df = df[df['loss_function'].isin(selected)]

    return df


In [ ]:
short_caption = 'Comparison of DSC, sensitivity and PPV ' + \
          'for models with various loss functions.'

caption = 'Comparison of DSC, sensitivity and PPV ' + \
          'for models with various loss functions. ' +\
          'These experiments used the UNet architecture and no augmentation.'

df_latex = loss_filter(df_results)[['loss_function_', 'include_T1_', 'mean dsc', 'median dsc', 'mean ppv', 'mean sensitivity']]

write_latex_multi_row(df_latex, table_name='loss_functions_summary', caption=caption, short_caption=short_caption, 
                         first_column = 'loss_function_', second_column = 'include_T1_')


In [ ]:
df_box_loss = filter_df_box(df, df_box, loss_filter)

In [ ]:
df_box_loss

In [ ]:
column_order = get_column_order(df_box_loss, x_column='loss_function', hue_column='include_T1')

In [ ]:
column_order

In [ ]:
short_caption = 'Voilin plot comparison of DSCs obtained with various loss functions ' + \
          'with the T2 image only or both the T1 and T2 images.'

caption = 'Voilin plot comparison of DSCs obtained with various loss functions ' + \
          'with the T2 image only or both the T1 and T2 images. ' + \
          'The dotted red line shows the mean baseline DSC of 0.81. ' +\
          'These experiments used the UNet architecture and no augmentation.'

order = get_column_order(df_box_loss)

plot_from_df(df=df_box_loss, 
             fname_stem='compare_loss_voilin',
             caption=caption,
             short_caption=short_caption,
             x_column='loss_function_trim', hue_col='include_T1_', 
             x_label='Loss Function', sns_plot='sns.violinplot', axhline=0.81, 
             set_ylim=(0.5, 1))



In [ ]:
short_caption = 'Boxplot comparison of DSCs obtained with various loss functions ' + \
          'with the T2 image only or both the T1 and T2 images.'

caption = 'Boxplot comparison of DSCs obtained with various loss functions ' + \
          'with the T2 image only or both the T1 and T2 images. ' + \
          'The dotted red line shows the median baseline DSC of 0.83. ' +\
          'These experiments used the UNet architecture and no augmentation.'

#fig_fname=fname_boxplot_T2,
plot_from_df(df=df_box_loss, 
             fname_stem='compare_loss_boxplot',
             caption=caption,             
             short_caption=short_caption,
             x_column='loss_function_trim', hue_col='include_T1_', 
             x_label='Loss Function', 
             sns_plot='sns.boxplot', axhline=0.83, 
             set_ylim=(0.5, 1))


In [ ]:
loss_filter(df)

In [ ]:
order = get_column_order(df_box_loss,x_column='loss_function', hue_column='include_T1')
df1 = loss_filter(df, selected=['TverskyLoss', 'TverskyFocalLoss', 'DiceLoss'])

for pid in order_of_patient_ids[:34]:

    short_caption = f'Comparison of models using different loss functions on subject number {pid}.'

    caption = f'Comparison of models using different loss functions on subject number {pid}. ' + \
              'The DSC achieved by each model is shown along with the number of ' + \
              'true positives/false negatives/false positives (in brackets). The baseline (top-right image) is the ' + \
              'performance of labeller two, assuming labeller one was the ground truth. Segmentations in the ' + \
              'left column use only the T2-weighted images. Segmentations in the centre column used both ' + \
              'T1 and T2-weighted images. ' +\
              'These experiments used the UNet and no augmentation.'    
    
    create_3col_figure(df1, patient_id=pid, fname_stem='compare_loss', caption=caption, short_caption=short_caption, \
                       hue_column = 'include_T1_', sort_column='loss_function_', \
                       verbose=True)

In [ ]:
order = get_column_order(df_box_loss,x_column='loss_function', hue_column='include_T1')
df1 = loss_filter(df, selected=order[3:])

for pid in order_of_patient_ids[:34]:

    short_caption = f'Comparison of models using different loss functions on subject number {pid}.'
    
    caption = f'Comparison of models using different loss functions on subject number {pid}. ' + \
              'The DSC achieved by each model is shown along with the number of ' + \
              'true positives/false negatives/false positives (in brackets). The baseline (top-right image) is the ' + \
              'performance of labeller two, assuming labeller one was the ground truth. Segmentations in the ' + \
              'left column use only the T2-weighted images. Segmentations in the centre column used both ' + \
              'T1 and T2-weighted images. ' +\
              'These experiments used the UNet and no augmentation.'    
    
    create_3col_figure(df1, patient_id=pid, fname_stem='compare_loss2', caption=caption, short_caption=short_caption, \
                       hue_column = 'include_T1_', sort_column='loss_function_', \
                       verbose=True)

In [ ]:
df_box_loss.groupby(['loss_function', 'include_T1'], as_index=False).mean()

In [ ]:
df_box.groupby(['loss_function', 'include_T1'], as_index=False).median()

## Networks experiment

In [ ]:
def network_filter(df, selected=[]):
    conditions =  ( df['loss_function'] == 'DiceLoss'         ) & \
                  ( df['augmentation'] == False          ) & \
                  ( True )
    df = df[conditions]
    if selected:
        df = df[df['loss_function'].isin(selected)]

    return df

In [ ]:
from utils.report import *

In [ ]:
df_box_network= filter_df_box(df, df_box, network_filter)

In [ ]:
short_caption = 'Comparison of DSC, sensitivity and PPV ' + \
          'for models with various network architectures.'

caption = 'Comparison of DSC, sensitivity and PPV ' + \
          'for models with various network architectures. ' +\
          'These experiments used the Dice loss function and no augmentation.'

df_latex = network_filter(df_results)[['network_name_', 'include_T1_', 'mean dsc', 'median dsc', 'mean ppv', 'mean sensitivity']]

write_latex_multi_row(df_latex, table_name='networks_summary', caption=caption, short_caption=short_caption, 
                         first_column = 'network_name_', second_column = 'include_T1_')


In [ ]:
column_order = get_column_order(df_box_network, x_column='network_name', hue_column='include_T1')

In [ ]:
column_order

In [ ]:
caption = 'Voilin plot comparison of DSCs obtained with various network architectures ' +\
          'with the T2 image only or both the T1 and T2 images. ' + \
          'The dotted red line shows the mean baseline DSC of 0.81. ' +\
          'These experiments used the Dice loss and no augmentation.'


#fig_fname=fname_boxplot_T2,
plot_from_df(df=df_box_network, \
             fname_stem='compare_networks_voilin',
             caption=caption,
             x_column='network_name_', hue_col='include_T1_', \
             x_label='Network', sns_plot='sns.violinplot', axhline=0.83,
             set_ylim=(0.5, 1))



In [ ]:
short_caption = 'Boxplot comparison of DSCs obtained with various network architectures ' + \
          'with the T2 image only or both the T1 and T2 images.'

caption = 'Boxplot comparison of DSCs obtained with various network architectures ' + \
          'with the T2 image only or both the T1 and T2 images. ' + \
          'The dotted red line shows the median baseline DSC of 0.83. ' +\
          'These experiments used the Dice loss and no augmentation.'


#fig_fname=fname_boxplot_T2,
plot_from_df(df=df_box_network, \
             fname_stem='compare_networks_boxplot',
             caption=caption, short_caption=short_caption,             
             x_column='network_name_', hue_col='include_T1_', \
             x_label='Network', sns_plot='sns.boxplot', axhline=0.83,
             set_ylim=(0.5, 1))


In [ ]:
df1 = network_filter(df, selected=['UNet_2Plus', 'UNet', 'UNet_3Plus'])

for pid in order_of_patient_ids[:34]:
    
    
    short_caption = f'Comparison of models using different network architectures on subject number {pid}.'
    
    caption = f'Comparison of models using different network architectures on subject number {pid}. ' + \
              'The DSC achieved by each model is shown along with the number of ' + \
              'true positives/false negatives/false positives (in brackets). The baseline (top-right image) is the ' + \
              'performance of labeller two, assuming labeller one was the ground truth. Segmentations in the ' + \
              'left column use only the T2-weighted images. Segmentations in the centre column used both ' + \
              'T1 and T2-weighted images. ' +\
              'These experiments used the Dice loss and no augmentation.'    
    
    create_3col_figure(df1, patient_id=pid, fname_stem='compare_networks', caption=caption, short_caption=short_caption, \
                       hue_column = 'include_T1_', sort_column = 'network_name_', \
                       verbose=True)


In [ ]:
df_box_network.groupby(['network_name', 'include_T1'], as_index=False).mean()

In [ ]:
df_box_network.groupby(['network_name', 'include_T1'], as_index=False).median()

# Print summary table

In [ ]:
#df_results.index.name = 'Exp. Id'
df_latex = df_results[['network_name_', 'loss_function_', 'include_T1_', 'augmentation_']]


In [ ]:
short_caption = 'Complete set of experiments configurations presented in this work.'
caption = 'Complete set of experiments configurations presented in this work. The base experiment setting uses the standard UNet, ' + \
          'the Dice loss function and no augmentation. ' #+ \
#           'Each experiment is given a unique experiment id (Exp. Id) ' + \
#           'which we refer to throughout.'

write_latex_table2D(df_latex, table_name='table_of_experiments', caption=caption, short_caption=short_caption, index=True)

In [ ]:
df_latex

# In depth treatment of best model F-Beta

In [ ]:
plot_density_plot_by_df_idx(df, df_box, df_idx=6, fname_stem='density_plots_fbeta', delta=0.05)

# Create 9x9 plots for appendix (by model idx)

In [ ]:
model_name_ = model_idx_to_name(df, 6)

short_caption = f'Predictions made by the standard UNet model on T2-weighted images (Dice loss, no augmentation).'

caption = f'Predictions made by the standard UNet model on T2-weighted images (Dice loss, no augmentation). ' + \
          'The DSC achieved by each model is shown along with the number of ' + \
          'true positives/false negatives/false positives (in brackets).'


best_order_of_patient_ids = get_pid_ordered_by_dsc(model_name_)[::-1]
idx = 0
while True:
    ids_subset = best_order_of_patient_ids[idx:idx+9]
        
    fname_stem = f'complete_model_results_fbeta_9x9_{str(idx)}'
    idx+=9
    
    create_3col_figure_by_model(patient_ids=ids_subset, model_name=model_name_, fname_stem=fname_stem, caption=caption, short_caption=short_caption)  

    if idx > len(best_order_of_patient_ids):
        break
    